In [ ]:
## 멜론 주간 차트 크롤링

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [9]:
# 한빛아카데미
url='https://www.hanbit.co.kr/academy/'
req=requests.get(url)
html=req.text
#html

In [10]:
# 멜론 주간차트
# 헤더에 User-Agent 값을 주어야만 데이터를 가져올 수 있음
url='https://www.melon.com/chart/week/index.htm'
header={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
req=requests.get(url,headers=header)
html=req.text
#html

In [7]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(html,'html.parser')

In [ ]:
## copy -> selector
#lst50 > td:nth-child(6) > div > div > div.ellipsis.rank02 > a

## copy -> Xpath
//*[@id="lst50"]/td[6]/div/div/div[2]/a

In [11]:
trs = soup.select('#lst50')
len(trs)

50

In [14]:
type(trs)

bs4.element.ResultSet

In [45]:
tr=trs[0]
#tr

In [20]:
# 순위 찾아오기
rank = tr.select_one('.rank').string       # .string 또는 .get_text()
rank = int(rank)
rank

1

In [26]:
# 곡명 찾아오기
title = tr.select_one('.ellipsis.rank01')
title

<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('19041301',33507137);" title="Butter 재생">Butter</a>
</span></div>

In [27]:
title = title.find('a').get_text()   # a 태그에서 가져옴
title

'Butter'

In [28]:
# 가수
artist = tr.select_one('.ellipsis.rank02')
artist

<div class="ellipsis rank02">
<a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a><span class="checkEllipsis" style="display:none"><a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a></span>
</div>

In [30]:
artist = artist.find('a').get_text()
artist

'방탄소년단'

In [34]:
# 앨범명
album = tr.select_one('.ellipsis.rank03')
album = album.find('a').string
album

'Butter'

- 50곡에 대해서 데이터프레임 만들기

In [35]:
rank_list, title_list, artist_list, album_list = [], [], [], []
for tr in trs:
    rank = tr.select_one('.rank').string
    rank = int(rank)

    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()
    
    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()

    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string
    
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)


In [37]:
df=pd.DataFrame({
    '순위' : rank_list,
    '곡명' : title_list,
    '가수' : artist_list,
    '앨범' : album_list
})

df.head()

,순위,곡명,가수,앨범
0,1,Butter,방탄소년단,Butter
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN
4,5,롤린 (Rollin'),브레이브걸스,Rollin'


- 이미지 속성값 가져오기

In [41]:
image = tr.select_one('.image_typeAll')    #tr=trs[0]
image

<a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('10612483');" title="Butter">
<img alt="Butter - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize" width="60">
<span class="bg_album_frame"></span>
</img></a>

In [42]:
image = image.find('img')
image

<img alt="Butter - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize" width="60">
<span class="bg_album_frame"></span>
</img>

In [44]:
image = image['src']
image

'https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize'

In [ ]:
# practice: lst50에서 50곡, lst100에서 50곡 합쳐서 100곡 데이터프레임 만들기 - rank, title, artist, album, image

In [55]:
trs = soup.select('#lst50')
len(trs)

50

In [48]:
trs2 = soup.select('#lst100')
len(trs2)

50

In [56]:
# for loop 한번에 하려면
trs.extend(trs2)
len(trs)

100

In [57]:
# 1 ~ 100위 가져오기

rank_list, title_list, artist_list, album_list, image_list = [], [], [], [], []

for tr in trs:
    rank = tr.select_one('.rank').string
    rank = int(rank)

    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()
    
    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()

    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string

    image = tr.select_one('.image_typeAll')
    image = image.find('img')
    image = image['src']
    
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    image_list.append(image)


In [58]:
df = pd.DataFrame({
    '순위' : rank_list,
    '곡명' : title_list,
    '가수' : artist_list,
    '앨범' : album_list,
    '이미지':image_list
})

df.head()

,순위,곡명,가수,앨범,이미지
0,1,Butter,방탄소년단,Butter,https://cdnimg.melon.co.kr/cm2/album/images/10...
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL,https://cdnimg.melon.co.kr/cm2/album/images/10...
2,3,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice,https://cdnimg.melon.co.kr/cm2/album/images/10...
3,4,헤픈 우연,헤이즈 (Heize),HAPPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...
4,5,롤린 (Rollin'),브레이브걸스,Rollin',https://cdnimg.melon.co.kr/cm2/album/images/10...


- 파일에 저장하기

In [61]:
period = soup.select_one('.yyyymmdd').get_text().strip()
period

'2021.05.24 ~ 2021.05.30'

- 100곡 모두 가져오기

In [62]:
filename = f'melon({period}).csv'
df.to_csv(filename, index=False, sep=',', encoding='utf8')